In [1]:
import sys
import os

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(os.path.join(project_root, 'src'))

from data_cleaning import *

# 🧼 Rensing og klargjøring av data

I denne notebooken klargjør vi de innhentede datasettene fra NILU og Frost for videre analyse og prediktiv modellering. Renseprosessen handler både om å forbedre datakvalitet, og om å tilpasse datastrukturen til videre bruk i visualisering, analyse og maskinlæring.

Vi utfører blant annet:
- Strukturering og transformasjon av data
- Fjerning av uteliggere og urealistiske verdier
- Behandling av manglende data
- Tidsserietilpasning og kvalitetssikring av måleperioder

All renselogikk er kapslet i egne Python-moduler for å gjøre notebooken ryddig og funksjonell. Det betyr at denne notebooken kun importerer og bruker funksjoner definert i `src/`, slik at det blir enkelt å gjenbruke og vedlikeholde koden.


### Om rensing og datakvalitet

Etter innhenting av rådata fra Frost (MET) og NILU, har vi gjennomført omfattende rensing for å sikre at datagrunnlaget er pålitelig og egnet for analyse og modellering. Renseprosessen har vært tilpasset egenskapene til hvert enkelt datasett.

Vi valgte to datasett med ulike utfordringer:

- **Frost**: Strukturert og nesten analyseklart, men med behov for validering og omstrukturering
- **NILU**: Reelt utfordrende – med manglende verdier, uteliggere og ujevn målefrekvens

Denne kontrasten har gitt oss anledning til å vise både enkel og avansert databehandling, samt reflektere rundt hvordan ulike strategier kreves for ulike datatyper.

---

## 🌫️ NILU – komplekst, men realistisk

Datasettet fra NILU inneholder daglige målinger for tre luftkomponenter i Trondheim. Vi valgte dette bevisst fordi det illustrerer virkelige utfordringer med miljødata, spesielt:

- Inkonsekvent dekning mellom komponenter
- Støy og uteliggere
- Ufullstendig tidsdekning

Vi utviklet en egen rensefil (`data_cleaning_nilu.py`) med tydelig struktur:

### 🔧 Rensing av NILU-data – steg for steg

1. **Konvertering og sortering av datoer**
   - Vi konverterer `dateTime` til `datetime`-objekter og sorterer datasettet kronologisk.
   - Dette er nødvendig for å kunne jobbe med dataserier og bruke KNN-imputasjon senere.

2. **Fjerning av lite relevante komponenter**
   - Kolonnen `Benzo(a)pyrene in PM10` fjernes fordi den består nesten utelukkende av manglende verdier og ikke er sentral for vår analyse.

3. **Outlier-fjerning**
   - Med `OutlierValidator` fjerner vi verdier som er mer enn 4 standardavvik fra gjennomsnittet.
   - Dette gjør vi separat for hver komponent (NO₂, PM10, PM2.5) for å redusere påvirkning fra ekstremverdier.

4. **KNN-imputasjon for manglende verdier**
   - Vi bruker `KNNImputer` fra `sklearn` til å estimere manglende verdier.
   - Imputasjonen baseres på lignende dager i datasettet, noe som gir mer realistiske estimater enn enkle gjennomsnitt.
   - Du kan lese mer om dette steget [**her**](../notebooks/KNN_imputation.ipynb).

5. **Merking av estimerte verdier**
   - Vi legger til egne kolonner (`generated_NO2`, `generated_PM10`, `generated_PM2.5`) som markerer hvilke verdier som er imputert med KNN.
   - Dette gir transparens og gjør det mulig å filtrere estimerte verdier i videre analyser.

6. **Datavalidering**
   - Vi bruker `MissingValueValidator`, `OutlierValidator` og `DateContinuityValidator` til å gi oversikt over datakvalitet før og etter rensing.
   - Dette gir dokumentasjon av renseprosessen og innsikt i effekten av hvert steg.



---

## 🌦️ Frost – robust, men ikke perfekt

Frost-dataene fra MET har høy kvalitet, men ble likevel renset og validert med `data_cleaning_frost.py`.

### 🔧 Håndtering av Frost-data – steg for steg

1. **Innlasting og datakonvertering**
   - Data hentes fra en rå `.json`-fil og konverteres til et strukturert DataFrame-format.
   - `referenceTime` konverteres til `datetime` og brukes som indeks for sortering.

2. **Filtrering av ønskede måleparametere**
   - Vi fokuserer på tre sentrale værvariabler:
     - Daglig gjennomsnittstemperatur (`mean_air_temperature`)
     - Daglig total nedbør (`sum_precipitation_amount`)
     - Daglig gjennomsnittlig vindstyrke (`mean_wind_speed`)
   - Alle andre måletyper ekskluderes.

3. **Pivotering og navneforenkling**
   - Datasettet transformeres slik at én rad representerer én dato, med egne kolonner for hver værkomponent.
   - Variabelnavn gjøres mer lesbare og konsistente.

4. **Validering og rensing av verdier**
   - Vi bruker `ValueRangeValidator` til å fjerne urealistiske målinger:
     - Temperatur: mellom -30 og 40 °C
     - Nedbør: mellom 0 og 250 mm
     - Vind: mellom 0 og 60 m/s
   - Dette hindrer feilaktige uteliggere i å forstyrre analysen.

5. **Kontinuitet i datoer**
   - `DateContinuityValidator` kontrollerer at det ikke er større hull i tidsserien.
   - Vi logger eventuelle manglende datoer, men imputasjon gjøres ikke da Frost-dataene er svært komplette.

6. **Lagring i SQLite**
   - Det rensede datasettet lagres i en SQLite-database (`frost.db`) for videre analyse og prediktiv modellering.
   - Dette gjør det enkelt å koble Frost- og NILU-data senere ved hjelp av SQL.


---

## 🔎 Hvorfor disse valgene?

Valgene våre er gjort med tanke på:
- **Datakvalitet**: Vi ønsker å kunne stole på analysene
- **Åpenhet**: Vi synliggjør estimerte data og ekstreme verdier
- **Brukervennlighet**: Formatene vi lagrer i er lette å bruke i både Python og SQL
- **Tilpasning til datasettene**: Frost og NILU krever ulik tilnærming – og det har vi tatt hensyn til

Ved å bruke egne validator-klasser (som `MissingValueValidator`, `OutlierValidator`, `DateContinuityValidator`, `ValueRangeValidator`) har vi laget en fleksibel og gjenbrukbar løsning som også kan skaleres til andre datatyper.

---

Vil du se koden som gjør alt dette? Ta en titt på:
- [`data_cleaning_nilu.py`](../src/data_cleaning/data_cleaning_nilu.py)
- [`data_cleaning_frost.py`](../src/data_cleaning/data_cleaning_frost.py)
- [`data_validators.py`](../src/data_cleaning/data_validators.py)



---

### 🌫️ Rensing av NILU luftkvalitetsdata

Nå skal vi kjøre selve koden som renser luftkvalitetsdataene fra NILU. Som beskrevet over, er dette datasettet mer utfordrende med:

- Manglende verdier som må håndteres
- Outliers som må identifiseres og fjernes
- Data som må restruktureres for analyse

Vi bruker funksjonen `main_dc_nilu()` som implementerer alle stegene i renseprosessen beskrevet over.

In [10]:
main_dc_nilu()


JSON-filen ble lastet vellykket.

Dataset informasjon:

Antall rader i datasettet: 5222

Antall outliers fjernet per verdi:
  - NO2: 13 outliers fjernet
  - PM10: 52 outliers fjernet
  - PM2.5: 45 outliers fjernet

Genererte verdier:
  - generated_NO2: 1268 genererte verdier
  - generated_PM10: 524 genererte verdier
  - generated_PM2.5: 532 genererte verdier

Konverterer data til JSON format...
Renset data lagret i '/Users/oliverroddesnes/Documents/Anvendt_programering/prosjekt_miljodataanalyse/data/clean/cleaned_data_nilu.json'

Data rensing fullført


### 🌦️ Rensing av Frost værdata

Nå skal vi rense værdataene hentet fra Frost API. Dette datasettet er langt mer strukturert enn NILU-dataene, men det krever fortsatt:

- Omstrukturering for å få én rad per dag
- Fjerning av urealistiske verdier via validering
- Kontroll av datakontinuitet over hele måleperioden

Vi bruker funksjonen `default_clean_frost_data()` som gjennomfører hele renseprosessen steg for steg – slik det er beskrevet over.


In [11]:
default_clean_frost_data(project_root)


Missing values detected:
mean_wind_speed:
- 2012: 1
- 2019: 1

No outliers detected

No date gaps detected

Generated values:
- mean_wind_speed: 2

Cleaned data saved to '/Users/oliverroddesnes/Documents/Anvendt_programering/prosjekt_miljodataanalyse/data/clean/frost.db' in the table 'weather_data'.



---

### 🧠 Hvorfor dette er en god tilnærming

Vi har valgt rensestrategier basert på datakildenes egenskaper:

- **NILU** krevde:
  - Komplett rekonstruering av datastrukturen
  - Avansert imputasjon og glatting
  - Transparens rundt hvilke verdier som er estimert

- **Frost** krevde:
  - Kontroll av gyldige verdier og datoer
  - Lett justering og tilrettelegging for videre analyse

Dette viser:
- Forståelse av hva som påvirker datakvalitet
- Evne til å tilpasse metoder etter utfordring
- Fokus på transparens og sporbarhet

Resultatet er et datasett som både er **ryddet og dokumentert**, og klart for videre analyse, visualisering og modellering.

---

## 📈 Neste steg: Analyse og visualisering

Nå som begge datasett er renset og klargjort, er vi klare for å utforske innholdet mer inngående. I neste notebook skal vi:

- Beregne statistiske mål som gjennomsnitt, median og standardavvik
- Undersøke sammenhenger og trender i luftkvalitet og værdata over tid
- Lage visuelle fremstillinger som gjør datamønstre lettere å forstå


### [**Videre til analyse og visualisering**](02_data_analysis_and_visualisation.ipynb)
##### [**Til samlesiden**](../docs/samleside.md)

